# CheXpert : A Large Chest X-Ray Dataset and Competition

This competition launched by the Stanford ML group aims at finding a prediction model which could perform as well as radiologist to find different pathologies thanks to chest X-Ray. The Dataset available to train our model is composed of 223,414 chest radiographs of 65,240 patients.

<img src="view1_frontal.jpg" title="X-Ray image of the dataset" width = 320/>

The website of the competition:
https://stanfordmlgroup.github.io/competitions/chexpert/

[Publication](https://arxiv.org/pdf/1901.07031.pdf) : Irvin, Jeremy, et al. "CheXpert: A Large Chest Radiograph Dataset with Uncertainty Labels and Expert Comparison." arXiv preprint arXiv:1901.07031 (2019).

Our goal is first to reproduce main results obtained in the related paper, published in January 2019.

In [111]:
!pip install -qU torchvision


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [8]:
!pip install pillow


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [9]:
!pip install requests


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

we used this open source https://www.kaggle.com/code/dnik007/pneumonia-detection-using-pytorch notebook.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [10]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()

In [11]:
import json
metadata = json.load(open("/opt/ml/metadata/resource-metadata.json","r"))
bucket = metadata["UserProfileName"]
bucket

'd4300856-e57e-4672-9f23-33074100ef98'

In [12]:
bucket

'd4300856-e57e-4672-9f23-33074100ef98'

In [13]:
from boto3 import client

conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
for key in conn.list_objects(Bucket=bucket)['Contents']:
    print(key['Key'])

labels/labels.csv
labels/rois.json


In [14]:
# Downloading Data from S3

import boto3
import botocore
from boto3 import client
import os
os.mkdir("data")
os.mkdir("labels")

BUCKET_NAME = bucket
c = 0
conn = client('s3') 
for key in conn.list_objects(Bucket=bucket)['Contents']:
    print(key['Key'])
    KEY = key['Key']
    s3 = boto3.resource('s3')
    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, f"{key['Key']}")
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise
    c = c+1

labels/labels.csv
labels/rois.json


In [15]:

# Dcm to Png
!pip install pydicom
import pydicom as dicom
import pydicom
import json
import os
from pathlib import Path
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom import dcmread
import os
import pickle
from PIL import Image
import numpy as np
from PIL import Image
import PIL

def read_xray(path, voi_lut=True, fix_monochrome=True):
    try:
        print("Converting to PNG .........................")
        dicom = dcmread(path, force=True)
        print(dicom.SOPInstanceUID, ">>>>>>", dicom.StudyInstanceUID, ">>>>>", dicom.SeriesInstanceUID)
        #if voi_lut:
        if voi_lut and len(dicom.get("VOILUTSequence", [])):
            data = apply_voi_lut(dicom.pixel_array, dicom)
        else:
            data = dicom.pixel_array
        if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data
        data = data - np.min(data)
        data = data / np.max(data)
        data = (data * 255).astype(np.uint8)
        return data,dicom.PatientName
    except Exception as e:
        print(e, ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        return "corrupt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.3 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [20]:
# convert dicoms to png and store in training_data_png

from glob import glob
files_ = glob('data/*.dcm',recursive=True)
try:
    os.mkdir("training_data_png")
except:
    pass
print(files_)
c = 0
for i in files_:
    try:
        print("\n>>> ",i)
        img,patientname = read_xray(i)
        with open("filename.pkl", 'wb') as f:
            pickle.dump(img, f)
        ims = pickle.load(open("filename.pkl", "rb"))
        norm = (ims.astype(np.float) - ims.min()) * 255.0 / (ims.max() - ims.min())
        filename = str(i).split("/")[1].split("_")[0]
        print("\n>> filename",filename)
        Image.fromarray(norm.astype(np.uint8)).save(f"training_data_png/{filename}.png")
        c = c + 1
        print(c, "Done")
    except Exception as e:
        print(e)

[]


### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [21]:
from glob import glob
import pandas as pd
labels = pd.read_csv("./labels/labels.csv",header=1)
print(labels)


       0 Case Number Patient ID Patient Name  \
0      1   Case 2512       Anon        AA480   
1      2   Case 2524       Anon        AA468   
2      3   Case 2523       Anon        AA469   
3      4   Case 2511       Anon        AA481   
4      5   Case 2532       Anon        AA460   
..   ...         ...        ...          ...   
576  577   Case 2914       Anon        AA078   
577  578   Case 2788       Anon        AA204   
578  579   Case 2716       Anon        AA276   
579  580   Case 2961       Anon        AA031   
580  581   Case 2587       Anon        AA405   

                                      StudyInstanceUID           User  \
0     1.2.392.200036.9107.307.10535.111053519020655253  sudo@carpl.ai   
1          1204.1178.185.1186.1188.1182.20160811160940  sudo@carpl.ai   
2           1196.1233.1132.12.1227.17.20170607104844.8  sudo@carpl.ai   
3        1108.111.1132.1105.1112.1231.20190204105804.6  sudo@carpl.ai   
4     1.2.392.200036.9107.307.10535.111053519013055079  su

In [23]:
files_ = glob('training_data_png/*.png',recursive=True)

for file in files_:
    sagemaker_session.upload_data(path=file  , bucket=bucket, key_prefix="data/train")

## Train
### Training script
The `classifier.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [24]:
!pygmentize code/classifier.py

import argparse
import json
import os
import numpy as np
import time
import sys
import csv
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as tfunc
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import Image
import torch.nn.functional as func

from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics
import random
import pandas as pd
from copy import deepcopy
import argparse
import json
import logging
import os
import sys
#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import cuda
from timeit import default_timer as timer
impo

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `classifier.py` script above.


In [25]:
!pip install --upgrade sagemaker==2.152.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.1/751.1 kB 10.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.6/636.6 kB 8.2 MB/s eta 0:00:0000:01
  Created wheel for sagemaker: filename=sagemaker-2.152.0-py2.py3-none-any.whl size=1007509 sha256=1c4428c07b5db3a84ab007499d1bc299f2d67a9c7bfe46f63b3e9c5616d587b0
  Stored in directory: /root/.cache/pip/wheels/e5/5b/a4/ad61c362425ff3ffeb666f9dc85274cb0d6271f7d897c976df
Successfully built sagemaker
  Attempting uninstall: tblib
    Found existing installation: tblib 1.6.0
    Uninstalling tblib-1.6.0:
      Successfully uninstalled tblib-1.6.0
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.3.0
    Uninstalling importlib-metadata-6.3.0:
      Successfully uninstalled importlib-metadata

In [26]:
from sagemaker.pytorch import PyTorch

In [27]:
!pip show sagemaker | grep Version

Version: 2.152.0


In [125]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="classifier.py",
    role=role,
    py_version="py38",
    framework_version="1.12.0",
    instance_count=1,
    instance_type="ml.m5.large",
    # instance_type="ml.g4dn.xlarge",
    hyperparameters={"epochs": 1, "backend": "gloo"},
    dependencies=['code/requirements.txt'],
    source_dir = "code",
    max_run=20,
    
)

In [126]:
estimator.fit(
    {
        "training":"s3://"+bucket+"/data/train" ,
        "testing":"s3://"+bucket+"/data/train",
        "validating":"s3://"+bucket+"/data/train"
        }
    )

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-06-19-07-29-41-964


2023-06-19 07:29:46 Starting - Starting the training job...
2023-06-19 07:29:59 Starting - Preparing the instances for training...
2023-06-19 07:30:44 Downloading - Downloading input data...
2023-06-19 07:31:19 Training - Downloading the training image......
2023-06-19 07:32:10 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-06-19 07:32:13,028 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-06-19 07:32:13,030 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-06-19 07:32:13,038 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-06-19 07:32:13,041 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-06-19 07:32:13,743 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:

## Host your model in Sagemaker
### Create endpoint
After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `classifier.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `classifier.py`. Here we will deploy the model to a single ```ml.m4.xlarge``` instance.

In [127]:
estimator.__dict__

{'framework_version': '1.12.0',
 'py_version': 'py38',
 'instance_count': 1,
 'instance_type': 'ml.m5.large',
 'keep_alive_period_in_seconds': None,
 'instance_groups': None,
 'volume_size': 30,
 'max_run': 20,
 'input_mode': 'File',
 'metric_definitions': None,
 'model_uri': None,
 'model_channel_name': 'model',
 'code_uri': None,
 'code_channel_name': 'code',
 'source_dir': 'code',
 'git_config': None,
 'container_log_level': 20,
 '_hyperparameters': {'epochs': 1,
  'backend': 'gloo',
  'sagemaker_submit_directory': 's3://sagemaker-ap-south-1-023180687239/pytorch-training-2023-06-19-07-29-41-964/source/sourcedir.tar.gz',
  'sagemaker_program': 'classifier.py',
  'sagemaker_container_log_level': 20,
  'sagemaker_job_name': 'pytorch-training-2023-06-19-07-29-41-964',
  'sagemaker_region': 'ap-south-1'},
 'code_location': None,
 'entry_point': 'classifier.py',
 'dependencies': ['code/requirements.txt'],
 'uploaded_code': UserCode(s3_prefix='s3://sagemaker-ap-south-1-023180687239/pytorch

In [128]:
sagemaker_job_name = estimator._hyperparameters["sagemaker_job_name"]

In [129]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.c5.2xlarge")

INFO:sagemaker:Creating model with name: pytorch-training-2023-06-19-07-33-06-581
INFO:sagemaker:Creating endpoint-config with name pytorch-training-2023-06-19-07-33-06-581
INFO:sagemaker:Creating endpoint with name pytorch-training-2023-06-19-07-33-06-581


----!

### Evaluate

You can use the test images to evalute the endpoint. The accuracy of the model depends on how many it is trained. 

In [130]:
predictor.__dict__

{'endpoint_name': 'pytorch-training-2023-06-19-07-33-06-581',
 'sagemaker_session': <sagemaker.session.Session at 0x7f8c5cea9f90>,
 'serializer': <sagemaker.serializers.NumpySerializer at 0x7f8c5e834a50>,
 'deserializer': <sagemaker.deserializers.NumpyDeserializer at 0x7f8c5e8348d0>,
 '_endpoint_config_name': None,
 '_model_names': None,
 '_context': None}

In [131]:
predictor.endpoint_name 

'pytorch-training-2023-06-19-07-33-06-581'

## Invoke endpoint and test your model in Sagemaker

In [132]:
import boto3

client = boto3.client('sagemaker-runtime')

custom_attributes = "c000b4f9-df62-4c85-a0bf-7c525f9104a4"  # An example of a trace ID.
endpoint_name = predictor.endpoint_name                                        # Your endpoint name.
content_type = "application/json"                                        # The MIME type of the input data in the request body.
accept = "application/json"                                              # The desired MIME type of the inference in the response.
payload = json.dumps({"url":"https://storage.googleapis.com/kaggle-datasets-images/17810/23340/c8372ebbe20b0f671c2f3c501ba51412/dataset-cover.jpeg?t=2018-03-24-19-05-18"})                                           # Payload for inference.
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    CustomAttributes=custom_attributes, 
    ContentType=content_type,
    Accept=accept,
    Body=payload
    )

print(response)   

{'ResponseMetadata': {'RequestId': 'bfbfcc2d-8a67-47e0-9ea0-042c6ebf2cf2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'bfbfcc2d-8a67-47e0-9ea0-042c6ebf2cf2', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Mon, 19 Jun 2023 07:35:47 GMT', 'content-type': 'application/json', 'content-length': '39587', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f8c56b270d0>}


In [133]:
from pprint import pprint

In [134]:
pprint(response) 

{'Body': <botocore.response.StreamingBody object at 0x7f8c56b270d0>,
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '39587',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 19 Jun 2023 07:35:47 GMT',
                                      'x-amzn-invoked-production-variant': 'AllTraffic',
                                      'x-amzn-requestid': 'bfbfcc2d-8a67-47e0-9ea0-042c6ebf2cf2'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'bfbfcc2d-8a67-47e0-9ea0-042c6ebf2cf2',
                      'RetryAttempts': 0}}


In [135]:
r = json.load(response["Body"])

In [136]:
r

{'response': {'findings': [{'name': '0', 'probability': 0.18336337804794312},
   {'name': '1', 'probability': 0.04427511617541313},
   {'name': '2', 'probability': 0.7756104469299316},
   {'name': '3', 'probability': 0.5073148012161255},
   {'name': '4', 'probability': 0.34910085797309875},
   {'name': '5', 'probability': 0.4374925494194031},
   {'name': '6', 'probability': 0.1157761737704277},
   {'name': '7', 'probability': 0.6701857447624207},
   {'name': '8', 'probability': 0.15392674505710602},
   {'name': '9', 'probability': 0.5106658339500427},
   {'name': '10', 'probability': 0.3848821520805359},
   {'name': '11', 'probability': 0.06862420588731766},
   {'name': '12', 'probability': 0.25629597902297974},
   {'name': '13', 'probability': 0.015060472302138805}],
  'rois': [{'finding_name': '0',
    'type': 'Freehand',
    'points': [[241.07142857142858, 62.60267857142858],
     [393.75000000000006, 62.60267857142858],
     [401.78571428571433, 64.29464285714286],
     [458.035714

## Push code to your S3 bucket

In [52]:
import boto3

In [ ]:
bucket_name = bucket
key = 'artifact/code'
from boto3 import client
s3 = client('s3')
BUCKET_NAME = bucket
DIR_NAME = 'code'
# Iterate through the files in the directory
for root, dirs, files in os.walk(DIR_NAME):
    for file in files:
        # Construct the full local path of the file
        local_path = os.path.join(root, file)
        # Construct the full S3 path of the file
        s3_path = os.path.join(root.replace(DIR_NAME, key), file)
        # Upload the file to S3
        s3.upload_file(local_path, BUCKET_NAME, s3_path)
        print(f'Uploaded {local_path} to s3://{BUCKET_NAME}/{s3_path}')

## Push model to your S3 bucket

In [ ]:
modelbucket = estimator.output_path.split("/")[2]

In [ ]:
try:
    os.mkdir("model")
except:
    pass
BUCKET_NAME = modelbucket
s3 = boto3.resource('s3')
s3.Bucket(BUCKET_NAME).download_file(sagemaker_job_name + "/output/model.tar.gz", "model/model.tar.gz")

In [ ]:
s3 = client('s3')
BUCKET_NAME = bucket
s3.upload_file("model/model.tar.gz", BUCKET_NAME, "model/model.tar.gz")